# CYBER Master

This project is to process the data of recorded voices and baseline measurements for the attachment style interaction with depression and anxiety, in a sense this will work a the ETL or pre-processing of the data.

This notebook has a single purpose expressed in the following steps:
- Gather the answers from the survey DB
- Process the answers by uniques and present a report of the demographics, attachment styles and HADS answers
- From each answer, get the recordings from blob storage.
- For each recording transcribe them and associate them to the answer
- Store the resulting data in the DB for further analysis

In [32]:
# -*- coding: utf-8 -*-

# Import the necessary libraries
import os
import json
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import subprocess
import requests
from typing import List

load_dotenv(find_dotenv())

CONSENT_SURVEY_ID = "8"
STUDY_SURVEY_ID = "7"
OLLAMA_VERSION = "v0.6"

QUESTIONS = {
    "en": {
        "1": "What’s happening in the picture?",
        "2": "What led up to that scene?",
        "3": "What the characters are thinking or feeling?",
        "4": "What might happen next?",
    },
    "es": {
        "1": "¿Qué está pasando en la imagen?",
        "2": "¿Qué eventos llevaron a esta escena?",
        "3": "¿Qué están sintiendo o pensando los personajes?",
        "4": "¿Qué ocurrirá a continuación?",
    },
}

IMAGE_KEYS = [
    "Image 1 - 1",
    "Image 1 - 2",
    "Image 1 - 3",
    "Image 1 - 4",
    "Image 2 - 1",
    "Image 2 - 2",
    "Image 2 - 3",
    "Image 2 - 4",
    "Image 3 - 1",
    "Image 3 - 2",
    "Image 3 - 3",
    "Image 3 - 4",
    "Image 4 - 1",
    "Image 4 - 2",
    "Image 4 - 3",
    "Image 4 - 4",
    "Image 5 -1",
    "Image 5 - 2",
    "Image 5 - 3",
    "Image 5 - 4",
    "Image 6 - 1",
    "Image 6 - 2",
    "Image 6 - 3",
    "Image 6 - 4",
    "Image 7 - 1",
    "Image 7 - 2",
    "Image 7 - 3",
    "Image 7 - 4",
    "Image 8 - 1",
    "Image 8 - 2",
    "Image 8 - 3",
    "Image 8 - 4",
]


def transcribe_audio(file_path, language="en"):
    # Use the local vosk via CLI to get the transcription
    result_file = file_path.replace(".wav", ".txt")
    subprocess.run(
        [
            "vosk-transcriber",
            "-i",
            file_path,
            "-l",
            language,
            "-o",
            result_file,
            "--log-level",
            "INFO",
        ]
    )
    # Read the transcription
    with open(result_file, "r") as transcription_file:
        transcription = transcription_file.read()
        return transcription


def request_llm_analysis(prompts: List[str], language="en"):
    url = "http://localhost:11434/api/generate"
    prompt = (f"Respond with the attachment style, a percentage for the likelihood of developing anxiety and a percentage of developing depression from the following answers,"
              + " give just one conscise answer with no explanations. Consider only the following answers:\n")
    prompt += "\n".join(prompts)
    data = {
        "prompt": prompt,
        "model": OLLAMA_VERSION,
    }
    response = requests.post(url, json=data)
    result = ""
    for line in response.text.split("\n"):
        if '"done": true' in line or not line:
            return result
        else:
            try:
                result += json.loads(line)["response"]
            except Exception as e:
                print(e)
                print(line)
                continue


    return result


def score_HADS(answers: dict) -> tuple:
    anxiety_items = {
        "HADS 1": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 5": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 6": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 8": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 9": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 12": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 13": {"0": 3, "1": 2, "2": 1, "3": 0},
    }
    depression_items = {
        "HADS 2": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 3": {"0": 0, "1": 1, "2": 2, "3": 3},
        "HADS 4": {"0": 3, "1": 2, "2": 1, "3": 0},
        "HADS 7": {"0": 3, "1": 2, "2": 1, "3": 0},
        "HADS 10": {"0": 3, "1": 2, "2": 1, "3": 0},
        "HADS 11": {"0": 3, "1": 2, "2": 1, "3": 0},
        "HADS 14": {"0": 0, "1": 1, "2": 2, "3": 3},
    }

    anxiety_score = 0
    depression_score = 0

    for item, answer in answers.items():
        if item in anxiety_items:
            anxiety_score += anxiety_items[item][answer]
        elif item in depression_items:
            depression_score += depression_items[item][answer]

    return anxiety_score, depression_score


def score_RQ(answers: dict) -> dict:
    anxiety_score = 0
    avoidance_score = 0

    styles = {
        "A": "Secure",
        "B": "Fearful",
        "C": "Preoccupied",
        "D": "Dismissing",
    }

    results = {}
    for item, answer in answers.items():
        if item == "RQ1":
            results["style"] = styles[answer]
        elif item == "RQ2":
            anxiety_score += int(answer)
            avoidance_score += int(answer)
        elif item == "RQ3":
            anxiety_score -= int(answer)
            avoidance_score -= int(answer)
        elif item == "RQ4":
            anxiety_score -= int(answer)
            avoidance_score += int(answer)
        elif item == "RQ5":
            anxiety_score += int(answer)
            avoidance_score -= int(answer)

    results["anxiety"] = anxiety_score
    results["avoidance"] = avoidance_score
    return results

def get_only_numbers(text: str) -> str:
    return ''.join(filter(str.isdigit, text))

## Connect to the DB

In [33]:
import mysql.connector

connection = mysql.connector.connect(
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    port=os.getenv("DB_PORT"),
)
cursor = connection.cursor()

# query the answers table
query = (
    f"SELECT * FROM surveys_answer where surveys_answer.survey_id = {STUDY_SURVEY_ID}"
)
cursor.execute(query)
answers = cursor.fetchall()

In [34]:
# Parse the answers
json_answers = []

for answer in answers:
    dict_answer = json.loads(json.loads(answer[1]))
    if dict_answer.get("response-uuid") not in [
        answer.get("response-uuid") for answer in json_answers
    ]:
        json_answers.append({"id": answer[0], **json.loads(json.loads(answer[1]))})
    else:
        for json_answer in json_answers:
            if json_answer.get("response-uuid") == dict_answer.get("response-uuid"):
                json_answer.update(json.loads(json.loads(answer[1])))
                break
full_answers = [answer for answer in json_answers if len(answer.keys()) == 62]
# get answers by language in a dictionary
answers_by_language = {}
answers_by_language = {
    "en": len([answer for answer in full_answers if answer["language"] == "en"]),
    "es": len([answer for answer in full_answers if answer["language"] == "es"]),
    "pt": len([answer for answer in full_answers if answer["language"] == "pt"]),
}
print(answers_by_language)

{'en': 0, 'es': 10, 'pt': 0}


In [35]:
# Get the recording info from the recording table
recordings_query = f"SELECT * FROM surveys_recording"  # where surveys_recording.answer_id in ({','.join([str(answer.get('id')) for answer in full_answers])})"
cursor.execute(recordings_query)
recordings = cursor.fetchall()
print(recordings[0])

(131, 'recordings/blob', None, 'en', '88bd9dcedc08411a800024e83806387c')


In [36]:
for answer in full_answers:
    for recording in recordings:
        for key, value in answer.items():
            if not isinstance(value, str) or len(value) < 4:
                continue
            if value.replace("-", "") != recording[-1]:
                continue
            answer[key] = f"{recording[1].replace('recordings/', '')}.wav"


print(full_answers[-1])

{'id': 573, 'language': 'es', 'response-uuid': 'ab7de4d7-3475-4c59-b345-f35312272095', 'Demographic - Age': '57', 'Demographic - Gender': 'female', 'Demographic - Education level': 'bachelor', 'Demographic - Relationship status': 'married', 'Demographic - Native speaker': 'yes', 'Demographic - speech diagnosis': 'no', 'Demographic - psychiatric disorder': 'no', 'Demographic - Substances': 'no', 'RQ1': 'A', 'RQ2': '6', 'RQ3': '1', 'RQ4': '2', 'RQ5': '2', 'HADS 1': '1', 'HADS 2': '0', 'HADS 3': '0', 'HADS 4': '3', 'HADS 5': '0', 'HADS 6': '0', 'HADS 7': '3', 'HADS 8': '0', 'HADS 9': '1', 'HADS 10': '3', 'HADS 11': '2', 'HADS 12': '0', 'HADS 13': '3', 'HADS 14': '3', 'Image 1 - 1': 'blob_IOMVHa8.wav', 'Image 1 - 2': 'blob_sECyzPP.wav', 'Image 1 - 3': 'blob_slb6iz5.wav', 'Image 1 - 4': 'blob_DQxVBcd.wav', 'Image 2 - 1': 'blob_2Vmp8iJ.wav', 'Image 2 - 2': 'blob_LPCZEan.wav', 'Image 2 - 3': 'blob_Ig0pubk.wav', 'Image 2 - 4': 'blob_3qVhU86.wav', 'Image 3 - 1': 'blob_jxf0b6n.wav', 'Image 3 - 2

## Getting the recordings per answer

In [37]:
from azure.storage.blob import BlobServiceClient

CONTAINER_NAME = "recordings"

blob_service = BlobServiceClient.from_connection_string(
    os.getenv("AZURE_CONNECTION_STRING")
)
local_path = "./recordings"
# create the folder if not alrady
if not os.path.exists(local_path):
    os.mkdir(local_path)

# Sanity check, listing the blobs in the container
container_client = blob_service.get_container_client(CONTAINER_NAME)
blob_list = []
for blob in container_client.list_blobs():
    if "blob" in blob.name:
        blob_list.append(blob.name)

In [38]:
# Download the blobs into the recordings folder if not already downloaded
for blob in blob_list:
    result_file_name = blob.split("/")[-1]
    result_file_name = f"{result_file_name}.wav"
    if result_file_name not in os.listdir(local_path):
        with open(file=f"{local_path}/{result_file_name}", mode="wb") as audio_file:
            download_stream = container_client.download_blob(blob)
            audio_file.write(download_stream.readall())

# list all the files in the folder
files = os.listdir(local_path)
print(len(files))

689


## Vosk

We are using [vosk](https://alphacephei.com/vosk/) to transcribe the audio into text, then assign it to the answer for further processing

In [39]:
for answer in full_answers:
    for key, value in answer.items():
        if not isinstance(value, str) or not value.endswith(".wav"):
            continue
        if value not in files:
            print(f"File {value} not found")
            continue
        if value.replace(".wav", ".txt") in os.listdir(local_path):
            print(f"File {value} already transcribed")
            answer[key] = open(
                f"{local_path}/{value.replace('.wav', '.txt')}", "r"
            ).read()
            continue
        answer[key] = transcribe_audio(
            f"{local_path}/{value}", language=answer["language"]
        )

File blob_nrnej2h.wav already transcribed
File blob_6Pu57EX.wav already transcribed
File blob_DSHsnaO.wav already transcribed
File blob_tLTvyIY.wav already transcribed
File blob_oQmoc6K.wav already transcribed
File blob_wVigQOC.wav already transcribed
File blob_81DcWTq.wav already transcribed
File blob_yiLAcqi.wav already transcribed
File blob_2ZzcIQ2.wav already transcribed
File blob_ijGVE3a.wav already transcribed
File blob_9nF4u9E.wav already transcribed
File blob_Kemd4qU.wav already transcribed
File blob_rbSy8qO.wav already transcribed
File blob_7J4aQii.wav already transcribed
File blob_PmIsMAa.wav already transcribed
File blob_hh9HFZm.wav already transcribed
File blob_7oAe8QB.wav already transcribed
File blob_V6S1D46.wav already transcribed
File blob_Pbfp6Ij.wav already transcribed
File blob_ttmhOBh.wav already transcribed
File blob_fpWJDLo.wav already transcribed
File blob_hmRuNX3.wav already transcribed
File blob_KRBmVTH.wav already transcribed
File blob_7tnXxx6.wav already tran

In [40]:
for answer in full_answers:
    for key, value in answer.items():
        if not isinstance(value, str) or not value.endswith(".wav"):
            continue
        if value not in files:
            print(f"File {value} not found")
            continue
        if value.replace(".wav", ".txt") in os.listdir(local_path):
            print(f"File {value} already transcribed")
            answer[key] = open(
                f"{local_path}/{value.replace('.wav', '.txt')}", "r"
            ).read()
            continue
        answer[key] = transcribe_audio(
            f"{local_path}/{value}", language=answer["language"]
        )

## Scoring the validated scales

We are using the HADS and RQ scales to determine the anxiety and depression, and the attachment style status accordingly.

### HADS - Hospital Anxiety and Depression Scale
A self report tool developed in 1983 by [Zigmond and Snaith](https://pubmed.ncbi.nlm.nih.gov/6880820/). It allows a dimensional scoring of the status of the patient and allows the better understanding of the psychiatric and psychological needs of a hospital patient.

The scale is composed of 14 items in a 4-item likert scale pointed from 0 to 3 and dedicates 7 items to depression and anxiety respectively. The scoring occurs by aggregating the values of each item and group, therefore, distributed as follows:
- Scores 0 to 7: Normal
- Scores 8 to 10: Borderline abnormal (at risk(?))
- Scores 11 - 21:  Abnormal (Usually requires a proper diagnosis and treatment)


### RQ - The Relationship Questionnaire
A self report questionnaire developed in 1991 by [Bartholomew & Horowitz](https://pubmed.ncbi.nlm.nih.gov/1920064/) composed of two segments with the objective of measuring the adult attachment styles. the styles are defined on two axis: Avoidance and anxiety.
The avoidance axis refers to the internal model of others and the anxiety to the internal model of self and usually presented as a cartesian plane, in which an adult may have a combination of both axis.
The four general cartesian regions in the adult attachment are:
**Secure**: Low avoidance - Low anxiety: The person considers others and themselves as trustworthy.
**Preoccupied**: Low avoidance - High anxiety: The person considers others are more trustworthy and generate a dependency due to internal low self-worth.
**Dismissing**: High avoidance - Low anxiety: The person has low trust on others but is self-reliant (opossite to the one above).
**Fearful**: High avoidance - High anxiety: The person does not trust others and has low self worth.

In this scale a person will first select a paragraph that reflects directly one of the styles above, then rates the agreement with each style prototype in a 7-item likert scales. From the answers in the likert scales the values on both axis are extracted and calculated as follows:
- **Model of self**: (secure + dissmissing) - (preocupied + fearful)
- **Model of others**: (secure + preocupied) - (dissmissing + fearful)

In [41]:
# Score HADS and RQ from the full_answers
for answer in full_answers:
    answer["HADS score - Anxiety"], answer["HADS score - Depression"] = score_HADS({f"HADS {i}": answer.get(f"HADS {i}") for i in range(1, 15)})
    answer["RQ score"] = score_RQ({f"RQ{i}": answer.get(f"RQ{i}") for i in range(1, 6)})


## Invoquing OLlama model using the prompts

The audio tracks were transcribed and will be fed in a custom prompt to a local Llama2 model using the [OLlama framework](https://ollama.com/)([Git hub documentation](https://github.com/ollama/ollama)).

The model has been pretrained by setting the `seed`, to ensure more deterministic results, and with a preparation prompt with extra instructions. The model is asked to act as a psiquiatrist assistant and given some context about the attachment styles, following the best practices mentioned in the literature ([1](https://arxiv.org/abs/2309.09128), [2](https://www.jmir.org/2023/1/e50638/), [3](https://link.springer.com/chapter/10.1007/978-981-99-7962-2_30)) The prompts and settings used in setup are available in the file `Modelfile` in this repository.


Then for each participant we agreggate all answers to the images and ask the model to determine the attachment style and two percentages for anxiety and depression afterwards.

In [42]:
# prepare the prompt to be fed to the LLM
for answer in full_answers:
    answer["prompts"] = []
    for key in IMAGE_KEYS:
        if answer.get(key) is None:
            continue
        answer.get("prompts").append(
            f"{key[0:8]} - {QUESTIONS.get(answer.get('language')).get(key[-1])} {answer[key]}"
        )

In [43]:
# request the LLM analysis

for answer in full_answers:
    answer["llm"] = []
    answer["llm_attachment_style"] = []
    answer["llm_anxiety"] = []
    answer["llm_depression"] = []
    for prompt in answer.get("prompts"):
        llm_answer = request_llm_analysis([prompt], language=answer.get("language"))
        answer.get("llm").append(llm_answer)
        for llm_ans in llm_answer.split("\n"):
            llm_ans_lower = llm_ans.lower()
            if "attachment style" in llm_ans_lower:
                style_name = ''.join(filter(str.isalpha, llm_ans_lower.replace("attachment style: ", "")))
                answer.get("llm_attachment_style").append(style_name)
            if len(llm_answer.split("\n")) == 1:
                answer.get("llm_anxiety").append(None)
                answer.get("llm_depression").append(None)
                print(f"Found an answer with no extra lines {llm_answer}")
                continue
            if "anxiety" in llm_ans_lower:
                answer.get("llm_anxiety").append(get_only_numbers(llm_ans_lower))
            if "depression" in llm_ans_lower:
                answer.get("llm_depression").append(get_only_numbers(llm_ans_lower))


Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful avoidant.
Found an answer with no extra lines Attachment style: Fearful Avoidant.
Found an answer with no extra lines Attachment style: Fearful av

## Store the resulting data

In [44]:
# save the answers in a json lines file
now_date = datetime.now().strftime("%Y-%m-%d--%H-%M")
with open(f"raw_answers-{now_date}.json", "w") as answers_file:
    answers_file.write(json.dumps(full_answers, indent=4))

## Final data preparation

In order to compare the results from RQ, HADS and the LLM data we need to transform the data from the LLM:
* __Attachment style:__ We want to report a percentage of each style to present from the amount of times one style is mentioned in the list.
* __Anxiety and Depression:__ An average value from the reported values may allow to compare to HADS.

In [45]:
for answer in full_answers:
    # get the llm_attachment_style, llm_anxiety, llm_depression
    llm_attachment_style = answer.get("llm_attachment_style")
    llm_anxiety = answer.get("llm_anxiety")
    llm_depression = answer.get("llm_depression")
    # calculate the average of the llm_anxiety and llm_depression
    anxiety_aux = [int(i) for i in llm_anxiety if i is not None]
    llm_anxiety_average = sum(anxiety_aux) / len(anxiety_aux) if len(anxiety_aux) > 0 else None
    depression_aux = [int(i) for i in llm_depression if i is not None]
    llm_depression_average = sum(depression_aux) / len(depression_aux) if len(depression_aux) > 0 else None
    answer["llm_anxiety_average"] = llm_anxiety_average
    answer["llm_depression_average"] = llm_depression_average
    # get the amount of times the attachment style was mentioned
    attachment_styles_aux = {}
    for style in llm_attachment_style:
        if style in attachment_styles_aux:
            attachment_styles_aux[style] += 1
        else:
            attachment_styles_aux[style] = 1
    # get the percentage of the attachment styles
    total = sum(attachment_styles_aux.values())
    attachment_styles = {k: v / total for k, v in attachment_styles_aux.items()}
    answer["llm_attachment_styles_percentages"] = attachment_styles



In [46]:
# save the answers in a json lines file
now_date = datetime.now().strftime("%Y-%m-%d--%H-%M")
with open(f"answers-{now_date}.json", "w") as answers_file:
    answers_file.write(json.dumps(full_answers, indent=4))

## Data analysis

We will start doing a simple exploratory analysis of the data, checking the amount of people and the distribution of the demographic data gathered from the surveys.
Then we will compare the results got from the HADS and RQ to the results from the LLM.

In [47]:
# Add the information into a Dataframe
import pandas as pd

df = pd.DataFrame(full_answers)
# describe the dataframe to get the statistics
# check only for the columns with the 'Demographic - ' prefix
demographics = df[
    [col for col in df.columns if col.startswith("Demographic - ")]
]
demographics.describe()

,Demographic - Age,Demographic - Gender,Demographic - Education level,Demographic - Relationship status,Demographic - Native speaker,Demographic - speech diagnosis,Demographic - psychiatric disorder,Demographic - Substances
count,10,10,10,10,10,10,10,10
unique,9,2,3,3,1,1,1,1
top,23,female,bachelor,single,yes,no,no,no
freq,2,5,6,4,10,10,10,10


In [48]:
demographics["Demographic - Gender"].value_counts()

Demographic - Gender
female    5
male      5
Name: count, dtype: int64

In [49]:
# Get the max and min age
demographics["Demographic - Age"].max(), demographics["Demographic - Age"].min()

('59', '23')

In [52]:
# get HADS scores and llm depression and anxiety averages
df["llm_depression_average"] = df["llm_depression_average"].apply(
    lambda x: int(x) if x is not None else None
)
df["llm_anxiety_average"] = df["llm_anxiety_average"].apply(
    lambda x: int(x) if x is not None else None
)

# print the columns with the HADS score and the llm depression and anxiety averages as a table
print(
    df[
        [
            "HADS score - Anxiety",
            "HADS score - Depression",
            "llm_depression_average",
            "llm_anxiety_average",
        ]
    ]
)


   HADS score - Anxiety  HADS score - Depression  llm_depression_average  \
0                     4                        6                      45   
1                     7                        7                      53   
2                     6                        8                      48   
3                     2                        5                      48   
4                     4                        7                      44   
5                     5                        6                      38   
6                     4                        6                      39   
7                     1                        9                      43   
8                     3                        6                      58   
9                     2                        4                      35   

   llm_anxiety_average  
0                   63  
1                   69  
2                   65  
3                   65  
4                   57  
5            